In [ ]:
import torch

from src.dataset import COCODataset, COCODataloader
from src.model import SegNet
from src.trainer import Trainer
from src.utils import load_config, download_coco_data, seed_everything
from src.visuals import Segmentor

print("Pytorch version:", torch.__version__)
print("CUDA enabled:", torch.cuda.is_available())

if torch.cuda.is_available():
    print("Device:", torch.cuda.get_device_name())

In [ ]:
CONFIG = load_config()
seed_everything(CONFIG.seed)
model = SegNet(in_channels=3, out_channels=len(CONFIG.classes))
segmentor = Segmentor(model)
CONFIG

In [ ]:
data = download_coco_data(config=CONFIG, split="train", max_samples=CONFIG.num_samples)
train_end = int(CONFIG.num_samples * 0.8)
val_start, val_end = train_end, train_end + int(CONFIG.num_samples * 0.1)
test_start = val_end
print(data)

# import fiftyone
# fiftyone.launch_app(data)

In [ ]:
train_dataset = COCODataset(data[:train_end])
image, masks = train_dataset[0]
segmentor.visualize_mask(image, masks)

In [ ]:
val_dataset = COCODataset(data[val_start:val_end])
image, masks = val_dataset[0]
segmentor.visualize_mask(image, masks)

In [ ]:
test_dataset = COCODataset(data[test_start:])
image, masks = test_dataset[0]
segmentor.visualize_mask(image, masks)

In [ ]:
train_dl = COCODataloader(train_dataset, batch_size=CONFIG.batch_size, pin_memory=True)
val_dl = COCODataloader(val_dataset, batch_size=CONFIG.batch_size, pin_memory=True)
test_dl = COCODataloader(test_dataset, batch_size=CONFIG.batch_size, pin_memory=True)

In [ ]:
trainer = Trainer(model, dataloaders=(train_dl, val_dl))

In [ ]:
trainer.fit()

In [ ]:
for dataloader in (train_dl, val_dl, test_dl):
    losses, iou, m_iou = trainer.eval(dataloader)
    trainer.log_losses(losses, phase="eval")
    trainer.log_iou(iou, m_iou)
    print("====================================")

In [ ]:
for i in range(5):
    image, _ = test_dataset[i]
    segmentor.segment(image)